In [2]:
"""IMPORTS"""
import numpy as np
from helpers import *
from implementations import *
from utils import *
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [4]:
# Loading raw data
data_path = 'dataset'
x_tr_raw, x_te_raw, y_tr_raw, train_ids, test_ids = load_csv_data(data_path, sub_sample=False)

In [6]:
dup = 3
deg = 5
lambda_ = 0.0002805263157894737

# Remove irrelevant columns
columns_to_remove = np.concatenate((range(1, 25), range(54, 56)))
x_tr_remove_col = np.delete(x_tr_raw, columns_to_remove, axis=1)
x_te_remove_col = np.delete(x_te_raw, columns_to_remove, axis=1)

# Treat don't know and refuse values
x_tr_dk = replace_dk_values_with_nan(x_tr_remove_col)
x_val_dk = replace_dk_values_with_nan(x_te_remove_col)

# Nan to mean
x_tr_no_nan, x_val_no_nan = nan_to_mean(x_tr_dk, x_val_dk)

# Remove columns with one unique value
x_tr_no_unique, x_val_no_unique = remove_uniquevalue_cols(x_tr_no_nan,x_val_no_nan)

# Standardize
x_tr_std, x_val_std = standardize(x_tr_no_unique, x_val_no_unique)

# Build polynomial expansion
x_poly_tr = build_poly(x_tr_std, deg)
x_poly_val = build_poly(x_val_std, deg)

# Duplicate 1 rows to balance dataset
x_tr_duplicated, y_tr_duplicated = duplicate_1rows(x_poly_tr, y_tr_raw, dup)
y_tr_full = y_tr_duplicated

# Adding a constant feature
tx = np.c_[np.ones((x_tr_duplicated.shape[0], 1)), x_tr_duplicated]

In [12]:
# Train the model 
w, loss = ridge_regression(y_tr_full, tx, lambda_)
print(f"loss = {loss}")

# Save optimal weights
np.savetxt("w.txt", w)